In [1]:
import import_ipynb
import logging
import numpy as np
from optparse import OptionParser
import os
import sys
from time import time
import matplotlib.pyplot as plt
import random  
from random import sample 
import pandas as pd
import pickle
import scipy

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures
from mlxtend.classifier import StackingClassifier
from sklearn.decomposition import PCA, NMF, KernelPCA, LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
import xgboost as xgb
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics

import My_Transformers
from sklearn.preprocessing import StandardScaler
# Global var
My_Transformers.TRAIN_ROW_COUNT = 7613
from My_Transformers import *
# from Word2Vec_Transformer import WVEmbeddingTransformer, Text_cleaner_transformer, EMB_VOCAB_SET, WV_EMBEDDINGS
import warnings
warnings.filterwarnings ("ignore")

VERSION = 5

importing Jupyter notebook from My_Transformers.ipynb


In [2]:
def pickleSave (obj, file):

    if VERSION != '':

        file, ext = os.path.splitext (file)
        file += "_v" + str (VERSION) + ext
    dirs = os.path.dirname (file) 
    if dirs:
        os.makedirs (dirs, exist_ok=True)
    with open (file, 'wb') as f:
        pickle.dump (obj, f)
    return

def unpickle (file):

    if VERSION != '':

        file, ext = os.path.splitext(file)
        file += "_v" + str(VERSION) + ext
    return pickle.load (open (file, 'rb'))

In [3]:
def join_tfidf_categoricalVals (df, y, Xstr_transformer, LB, LB_loc):
    
    Xstr = df['text']
    if y is not None:
        X = scipy.sparse.hstack ((Xstr_transformer.fit_transform (Xstr, y), LB.transform (list (df['keyword'])),
                                  LB_loc.transform (list (pd.isnull (df['location'])))))
    else:
        X = scipy.sparse.hstack ((Xstr_transformer.transform (Xstr), LB.transform (list (df['keyword'])),
                                  LB_loc.transform (list (pd.isnull (df['location'])))))
    X = X.tocsr ()
    return X

In [4]:
def init ():

    train_file = "Data/all.csv"
    df = pd.read_csv (train_file)
    # df = df[0:20]
    df['text'] = df.text.astype (str)
    df['keyword'] = df.keyword.astype (str)
    keywords_train = list (df['keyword'])
    LB = LabelBinarizer ()
    LB.fit (keywords_train)
    LB_loc = LabelBinarizer ()
    LB_loc.fit (list (pd.isnull (df['location'])))
    Xstr = df["text"]
    y = df["target"]
    return df, y, LB, LB_loc


In [5]:
def train (df, y, LB, LB_loc, isCV=False):

    Xstr_transformer = Pipeline ([

        ('tfidf_vectorizer',        TfidfVectorizer (analyzer='char', ngram_range=(3, 7), max_df=0.90, min_df=5, token_pattern='(\S+)')),
        ('sparse_svm_feat_select1', SparseSVM_feature_selector ()),
        ('interactions',            PolynomialFeatures (2, interaction_only=True, include_bias=False)),
        ('sparse_svm_feat_select2', SparseSVM_feature_selector ()),
        ('standardization',         StandardScaler (with_mean=False)),
        ('kpca',                    KPCA_TrainTest_FeatureTransformer (kernel=['cosine'], isSparseOut=True))
    ])
    X = join_tfidf_categoricalVals (df, y, Xstr_transformer, LB, LB_loc)
    selectKBest_feature_selector = SelectKBest_feature_selector (score_func=[mutual_info_classif])
    X = selectKBest_feature_selector.fit_transform (X, y)

    pickleSave (X, 'X.bin')
    # X = unpickle ('X.bin')
    X.shape

    pickleSave (Xstr_transformer, 'Xstr_transformer.bin')
    # Xstr_transformer = unpickle ('Xstr_transformer.bin')
    # pickleSave (rf_importance_feature_selector, 'rf_importance_feature_selector.bin')

    params_lsvc  = { 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'},                                                                   'param_grid': {'C' : [0.005, 0.01, 0.05, 0.1, 1, 10]} }
    params_logit = { 'name': 'Logit', 'params': {'penalty': 'l2', 'class_weight': 'balanced'},                                                'param_grid': {'C': [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10], 'fit_intercept': [True, False]} }
    params_rf    = { 'name': 'RF',    'params': {'class_weight': 'balanced'} }
    params_xgb   = { 'name': 'Xgb',   'param_grid':{'max_depth': [3, 5], 'learning_rate':[1.5, 3]} }
    params_catb  = { 'name': 'Catb',  'isCV': isCV }

    params    = [params_lsvc, params_logit, params_catb]
    bclfs     = []
    cv_scores = []
    for param in params:

        bclf = Best_clf_cv_transformer (param)
        bclf.fit (X, y)
        bclfs.append ((bclf, bclf.get_cv_score()))
        cv_scores.append (bclf.get_cv_score())
    return X, Xstr_transformer, bclfs, params, selectKBest_feature_selector

In [6]:
def predict (bclfs, Xstr_transformer, LB, LB_loc, selectKBest_feature_selector):

    test_file = "Data/test.csv"
    testDF = pd.read_csv (test_file)
    # testDF = testDF[0:10]
    testDF['text'] = testDF.text.astype (str)
    testDF['keyword'] = testDF.keyword.astype (str)
    X_test_str = testDF["text"]
    testDF.shape

    X_test = join_tfidf_categoricalVals (testDF, None, Xstr_transformer, LB, LB_loc)
    X_test = selectKBest_feature_selector.transform (X_test)
    pickleSave (X_test, 'X_test.bin')
    X_test.shape
    for bclf in bclfs:

        bclf = bclf[0]
        predictions = bclf.predict (X_test)
        testDF['target'] = predictions
        testDF['target'] = testDF.target.astype (int)
        submitDF = testDF[['id', 'target']]
        submitDF.to_csv ('submission_v'+str (VERSION)+'_' + bclf.name + '.csv', index=False)
    return

In [7]:
def add_voting_clf (X, y, params, bclfs):

    # voting classifier
    a = 1.0 / np.sum (y)
    b = 1.0 / (len(y) - np.sum (y))
    w1 = a / (a + b)
    w0 = b / (a + b)
    sample_weight = []
    for i in y:
        if i:
            sample_weight.append (w1)
        else:
            sample_weight.append (w0)
    voting_candidates = []
    for param in params:

        param['isCV'] = False
        bclf = Best_clf_cv_transformer (param)
        voting_candidates.append ((bclf.name, bclf))
    voting_clf = VotingClassifier (estimators=voting_candidates, voting='soft', weights=[1, 1, 1.5])
    voting_clf.name = 'Voting'
    train_len = len (y[pd.isnull (y) == False])
    X_train, y_train = X[:train_len], y[:train_len]
    voting_clf_cv_score = 0
    print ('Voting_clf cv-score =', voting_clf_cv_score)
    voting_clf.fit (X_train, y_train)
    bclfs.append ((voting_clf, voting_clf_cv_score))
    bclf = max (bclfs, key=lambda i: i[1])[0]
    print ('bclf =', bclf)
    return bclfs

In [8]:
def main (threshold=0.9, maxIterCount=10, thresh_depreciation=0.01):

    X, params, bclfs, selectKBest_feature_selector = None, None, None, None
    df, y, LB, LB_loc = init ()
    newTrainAddCount = 1
    iterCount = 0
    while newTrainAddCount > 0:

        threshold = threshold - iterCount * thresh_depreciation
        iterCount += 1
        if iterCount > maxIterCount:
            break
        X, Xstr_transformer, bclfs, params, selectKBest_feature_selector = train (df, y, LB, LB_loc, False)
        bclf = bclfs[2][0]
        newTrainAddCount = 0
        for idx, row in df.iterrows ():
            if not pd.isnull (row['target']):

                continue
            print ('considering idx =', idx)
            x  = X[idx].reshape (1, -1)
            pr = bclf.predict_proba (x)[0]
            p  = 0
            if pr[0] > 0.5:
                pr = pr[0]
                p  = 0
            else:
                pr = pr[1]
                p  = 1
            if pr >= threshold:
                # add in the training set
                y[idx] = p
                df.at[idx, 'target'] = p
                newTrainAddCount += 1
        # arrange df, y such that 1st rows with y=0/1 then rows with y=NaN
        arrangedDF1 = df[pd.isnull(df['target']) == False]
        arrangedDF2 = df[pd.isnull(df['target'])]
        df = pd.concat ([arrangedDF1, arrangedDF2], axis=0)
        y = df['target']
    df.to_csv ('Data/all.csv', index=False)
    X = join_tfidf_categoricalVals (df, None, Xstr_transformer, LB, LB_loc)
    X = selectKBest_feature_selector.transform (X)
    bclfs = add_voting_clf (X, y, params, bclfs)
    predict (bclfs, Xstr_transformer, LB, LB_loc, selectKBest_feature_selector)
    # free memory
    df = None

    # CV for the bclfs on the original train rows
    print ('------------------- CV for the bclfs on the original train rows -------------------')
    X = X[:7613]
    y = y[:7613]
    cv_results = []
    bclf_cvscore = []
    for bclf in bclfs:

        bclf = bclf[0]
        cv_result = cross_validate (bclf, X, y, cv=5)
        cv_results.append ((bclf.name, cv_result))
        bclf_cvscore.append ((bclf, np.mean (cv_result['test_score'])))
    pickleSave (bclf_cvscore, 'bclf_cvscore.bin')
    for cv_result in cv_results:

        clf_name = cv_result[0]
        cv_result = cv_result[1]
        cv_score = np.mean (cv_result['test_score'])
        print ()
        print (clf_name, cv_score)
        print (cv_result)
    return

In [9]:
if __name__=="__main__":

    # main (maxIterCount=1)

SyntaxError: unexpected EOF while parsing (<ipython-input-9-49067fc9e82e>, line 3)


# Training snippet:


LinearSVC with L1-based feature selection for X.shape = (7613, 131984)

--------------------- Best parameter (CV score=0.678):

{'feature_selection__estimator__tol': 0.0001}

New #features =  1081

LinearSVC with L1-based feature selection for X.shape = (7613, 584821)

--------------------- Best parameter (CV score=0.765):

{'feature_selection__estimator__tol': 1e-05}

New #features =  1064

Kernel PCA features for X.shape = (10876, 1064)

--------------------- Best:  parameters = ('cosine', 0.1) , CV = 0.8331841246312564

KPCA dim = 1017

Find K-best features based on Mutual-Info / Chi^2 for X.shape = (7613, 1240)

--------------------- Best parameter (CV score=0.828):

{'classifier__C': 0.1, 'selectKBest__k': 992, 'selectKBest__score_func': <function mutual_info_classif at 0x0000023E651AB7B8>}

training LSvc for X.shape = (7613, 992)

LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.831):

{'C': 0.1}

training Logit for X.shape = (7613, 992)

Logit : Best_clf_cv_transformer: Best parameter (CV score=0.832):

{'C': 1, 'fit_intercept': True}

training Catb for X.shape = (7613, 992)


# Findings: 5-fold CV on train data:
        
LSvc 0.8245136552709529

{'fit_time': array([2.88406372, 2.83506417, 2.81396937, 2.81099486, 2.89106369]), 'score_time': array([0.02500296, 0.02500367, 0.02496219, 0.02500105, 0.02500272]), 'test_score': array([0.82731451, 0.78791858, 0.8358503 , 0.83968463, 0.83180026])}


Logit 0.8302943995830899

{'fit_time': array([0.59106588, 0.60597157, 0.61502814, 0.61699843, 0.53600168]), 'score_time': array([0.00600028, 0.00600433, 0.00599933, 0.00600028, 0.0060029 ]), 'test_score': array([0.8312541 , 0.79382797, 0.8345371 , 0.83968463, 0.8521682 ])}


Catb 0.7904926044194881

{'fit_time': array([149.75626206, 150.10961509, 147.45200038, 140.60964346,
       145.88396621]), 'score_time': array([0.11396813, 0.11701083, 0.10603738, 0.10497499, 0.11903477]), 'test_score': array([0.79908076, 0.76099803, 0.79120158, 0.77003942, 0.83114323])}


Voting 0.8225435136923718

{'fit_time': array([155.16131639, 162.68506241, 164.21599936, 148.20703459,
       153.78440833]), 'score_time': array([0.14597344, 0.1499753 , 0.14596581, 0.14196539, 0.14703369]), 'test_score': array([0.82403152, 0.79317137, 0.82665791, 0.81997372, 0.84888305])}